In [2]:
!pip install pytorch-lightning > /dev/null
!pip install torchmetrics > /dev/null
!pip install comet-ml > /dev/null
!pip install torch-ema > /dev/null
!pip install transformers > /dev/null
!pip install datasets > /dev/null

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from datasets import load_dataset

dataset_rewiew = load_dataset("Romjiik/Russian_bank_reviews")
dataset_alpha = load_dataset("ZhenDOS/alpha_bank_data")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [36]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("zaaabik/t5-russian-summarization-detox-finetuning")
model = AutoModelForSeq2SeqLM.from_pretrained("zaaabik/t5-russian-summarization-detox-finetuning")

In [51]:
def tokenize_function(examples):
    return tokenizer(examples["context"], padding="max_length", truncation=True)

In [52]:
tokenized_dataset_alpha = dataset_alpha.map(tokenize_function, batched=True)

In [53]:
next(iter(tokenized_dataset_alpha["train"])).keys()

dict_keys(['context', 'response', 'input_ids', 'attention_mask'])

In [60]:
item = next(iter(tokenized_dataset_alpha["train"]))
torch.Tensor(item["input_ids"]).int()

tensor([  183,    35,  3574,   909,  6732,    34, 23842,  1470,     3,   526,
          142,    35,  3841,  4803,     6,     8, 22506,    14, 28665,    13,
           25,     2], dtype=torch.int32)

In [75]:
import torch
model.encoder(input_ids=torch.Tensor(item["input_ids"]).int()[None, ...]).last_hidden_state.mean(dim=1).squeeze().shape

torch.Size([768])

In [76]:
def create_embedding(item):
  return {"context_emb" : model.encoder(input_ids=torch.Tensor(item["input_ids"]).int()[None, ...]).last_hidden_state.mean(dim=1).squeeze()}

In [79]:
embedded_dataset_alpha = tokenized_dataset_alpha.map(create_embedding)

Map:   0%|          | 0/270 [00:00<?, ? examples/s]

In [80]:
embedded_dataset_alpha

DatasetDict({
    train: Dataset({
        features: ['context', 'response', 'input_ids', 'attention_mask', 'context_emb'],
        num_rows: 270
    })
})

In [ ]:
embedded_dataset_alpha["train"]["context_emb"]

In [118]:
from sklearn.cluster import DBSCAN

clust_res = DBSCAN(eps=0.15, metric="cosine").fit_predict(embedded_dataset_alpha["train"]["context_emb"])

In [114]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [102]:
import numpy as np
view = TSNE(2).fit_transform(np.array(embedded_dataset_alpha["train"]["context_emb"]))

In [103]:
view.shape

(270, 2)

In [ ]:
import pandas as pd
pd.DataFrame(embedded_dataset_alpha["train"]["response"])[clust_res==2]

In [ ]:
clust_res

In [ ]:
plt.scatter(view[:, 1], view[:, 0], c=clust_res)